In [ ]:
import glob
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from lib.getDensity import getFilteredDensity
import src.utils as utils
import pickle
from scipy.io import savemat

In [1]:
if __name__ == "__main__":
    folders = np.array(range(33))
    ux = 53.69132701153139
    uy = 56.463734106758785
    enableShear = True
    outputshape = (111, 111)
    feature = "raw"  # "raw" "cart" "mlp"
    polyExtend = (
        1, 1
    )  # first is for coordinate poly order, second is for density poly order
    path = "flowdata1"
    exportPath = os.path.join("data", "densityResult")
    if feature == "mlp":
        polyExtend = (0, 0)
        regressor = MLPRegressor(hidden_layer_sizes=(
            100,
            100,
            100,
            10,
            10,
        ),
                                 early_stopping=True,
                                 max_iter=500)
    else:
        regressor = LinearRegression()
        # regressor = RidgeCV()
        # regressor = LassoCV()
    total_densityList = []
    total_depthList = []
    exportList = []
    densityMax = -1e9
    densityMin = 1e9
    depthList = []
    densityList = []
    flow_file_list = sorted(glob.glob(
            os.path.join(path,"*.npy")))
    h, w, _ = np.load(flow_file_list[0]).shape
    croppedShape = (min(h, w), min(h, w))
    x = np.linspace(0.0,
                        croppedShape[1] - 30,
                        croppedShape[1] - 30,
                        endpoint=False)
    y = np.linspace(0.0,
                        croppedShape[0] - 30,
                        croppedShape[0] - 30,
                        endpoint=False)
    axis = np.array(np.meshgrid(x, y)).transpose((1, 2, 0))
    for i in range(len(flow_file_list)):
            flow = np.load(flow_file_list[i])
            if h > w:
                flow = flow[(flow.shape[0] - croppedShape[0]) //
                            2:-(flow.shape[0] - croppedShape[0]) //
                            2, :, :].copy()
            elif h < w:
                flow = flow[:, (flow.shape[1] - croppedShape[1]) //
                            2:-(flow.shape[1] - croppedShape[1]) //
                            2, :].copy()
            
            density, zero_level = getFilteredDensity(flow, use_cuda=True)
            # remapping
            """
            density = (density + 1.0) / 1.0
            density = (density * 255.0).astype('uint8')

            density = density[20:-20, 20:-20]

            """
            map = (axis + flow[15:-15, 15:-15, :]).astype(np.float32)
            density = cv2.remap(density,
                                map[:, :, 0],
                                map[:, :, 1],
                                interpolation=cv2.INTER_LINEAR,
                                borderValue=float(zero_level))
            # resizing
            density = cv2.resize(density, (outputshape[1], outputshape[0]),
                                 interpolation=cv2.INTER_CUBIC)
            
            """
            loc_ind = i // 46
            # if loc_ind == 8 or loc_ind == 1 or loc_ind == 2 or loc_ind == 3 or loc_ind == 4 or loc_ind == 5 or loc_ind == 6 or loc_ind == 7:
            #     continue
            indent_ind = i - loc_ind * 46
            max_loc = np.argmax(density)
            max_y = max_loc // density.shape[1]
            max_x = max_loc - max_y * density.shape[1]
            """
            



            densityList.append(density.max())

            if i == 0:
                depthList.append(0.0)
            elif i == 1 or i == 5 or i == 9 or i == 13 or i == 17 or i == 21 or i == 25 or i == 29 or i == 33 or i == 37 or i == 41 or i == 45 or i == 49 or i == 53 or i ==57:
                depthList.append(0.5)
            elif i == 2 or i == 6 or i == 10 or i == 14 or i == 18 or i == 22 or i == 26 or i == 30 or i == 34 or i == 38 or i == 42 or i == 46 or i == 50 or i == 54 or i ==58:
                depthList.append(1.0)
            elif i == 3 or i == 7 or i == 11 or i == 15 or i== 19 or i == 23 or i == 27 or i == 31 or i == 35 or i == 39 or i == 43 or i == 47 or i == 51 or i == 55 or i ==59:
                depthList.append(1.5)
            elif i ==4 or i == 8 or i == 12 or i == 16 or i == 20 or i ==24 or i == 28 or i == 32 or i == 36 or i == 40 or i == 44 or i == 48 or i == 52 or i == 56 or i ==60:
                depthList.append(2.0)
            elif i>=61 and i<=70:
                depthList.append((i-60)*0.2)
            elif i>=71 and i<=80:
                depthList.append((i-70)*0.2)
            elif i>=81 and i<=90:
                depthList.append((i-80)*0.2)
            elif i>=91 and i<=100:
                depthList.append((i-90)*0.2)
            elif i>=101 and i<=110:
                depthList.append((i-100)*0.2)
            """
            if indent_ind == 0:
                if len(depthList) != 0:
                    # plt.plot(np.array(densityList)[:, -1],
                    #          depthList,
                    #          'x',
                    #          label=str(folder))
                    total_densityList.extend(densityList)
                    total_depthList.extend(depthList)
                depthList = []
                densityList = []
                depthList.append(0.0)
                if feature == "raw":
                    densityList.append((density.max()))
                elif feature == "cart":
                    densityList.append(
                        (abs(max_x - ux), abs(max_y - uy), density.max()))
                elif feature == "mlp":
                    densityList.append((max_x, max_y, density.max()))
                continue
            indent_level = ((indent_ind - 1) // 9 + 1.0) * 0.5
            if not enableShear and (indent_ind -
                                    1) % 9 != 0:  # skip shear step
                continue
            else:
                depthList.append(indent_level)
                if feature == "raw":
                    densityList.append((density.max()))
                elif feature == "cart":
                    densityList.append(
                        (abs(max_x - ux), abs(max_y - uy), density.max()))
                elif feature == "mlp":
                    densityList.append((max_x, max_y, density.max()))
                exportList.append(density)
                if density.max() > densityMax:
                    densityMax = density.max()
                if density.min() < densityMin:
                    densityMin = density.min()
                # print("File: ", folder, " Row: ", max_y, " column: ", max_x,
                #       " r: ", r)
                # plt.imshow(
                #     cv2.applyColorMap(((density - density.min()) /
                #                        (density.max() - density.min()) *
                #                        255.0).astype('uint8'),
                #                       cv2.COLORMAP_HOT))
                # plt.show(block=True)
                """
    #plt.plot(np.array(densityList)[:, -1], depthList, 'x', label=str(folder))
    total_densityList.extend(densityList)
    total_depthList.extend(depthList)
    # plt.legend()
    # plt.savefig("regress.jpg")
    X = np.array(total_densityList)
    if len(X.shape) == 1:
        X = X.reshape((-1, 1))
    # s = StandardScaler()
    # X = s.fit_transform(X)
    y = np.array(total_depthList)
    savemat(os.path.join(exportPath, "density2matlab"), {'X': X, 'y': y})
    if polyExtend[0] > 1 or polyExtend[1] > 1:
        poly1 = PolynomialFeatures(polyExtend[0], include_bias=False)
        poly2 = PolynomialFeatures(polyExtend[1], include_bias=False)
        if X.shape[1] == 1:
            X = poly2.fit_transform(X)
        else:
            X = np.hstack(
                (poly1.fit_transform(X[:, :-1]), poly2.fit_transform(X[:,
                                                                       -1:])))
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.2,
                                                        random_state=42)
    regressor.fit(X_train, y_train)
    print("Model type: ", type(regressor), " feature order: ", polyExtend)
    print("Mean test error: ",
          np.abs(regressor.predict(X_test) - y_test).mean())
    plt.plot(X_train, y_train , 'x')
    plt.plot(X_test,regressor.predict(X_test),color='black',label='best line')
    plt.savefig("prediction.jpg")
    if feature != "mlp":
        print(regressor.coef_, regressor.intercept_)
    with open('depthPredictor.pkl', 'wb') as f:
        pickle.dump((regressor, feature, polyExtend), f)
    # for i in range(len(exportList)):
    #     im = (exportList[i] - densityMin) / (densityMax - densityMin) * 255.0
    #     im = cv2.applyColorMap(im.astype('uint8'), cv2.COLORMAP_HOT)
    #     cv2.imwrite(os.path.join(exportPath, str(i) + ".jpg"), im)


NameError: name 'np' is not defined

In [1]:
with open('depthPredictor.pkl','rb') as f:

    model = pickle.load(f)

test = np.array(1.65)

test.resize(1,1)

regressor1 = model[0]

regressor1.predict(test)
 


NameError: name 'pickle' is not defined

In [ ]:
x = np.linspace(0,
                                19,
                                20,
                                dtype='float32')
y = np.linspace(0,
                                19,
                                20,
                                dtype='float32')
map = np.array(np.meshgrid(x, y)).transpose((1, 2, 0))